In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import time

import numpy as np
import pandas as pd

In [2]:
art_list = [1,4,4,2,3,1]
rand_list = [1,2,3,4,5]
art_list+=rand_list
art_list

[1, 4, 4, 2, 3, 1, 1, 2, 3, 4, 5]

In [3]:
art_list = []

In [7]:
# Set up options to use a custom User-Agent
chrome_options = Options()
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

# Set up the WebDriver (in this case for Chrome)
# You need to specify the path to your ChromeDriver
chrome_driver_path = r"/Users/ianchang/Library/Mobile Documents/com~apple~CloudDocs/1. Project/chromedriver-mac-x64/chromedriver"

# Initialize the WebDriver
service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=service)

# for n in range(1,5):
#     # Open the website
#     url = f'https://www.artnet.com/auctions/all-artworks/{n}'
#     driver.get(url)

#     time.sleep(5)

#     # Parse the page source using BeautifulSoup
#     soup = BeautifulSoup(driver.page_source, 'html.parser')

#     # Extract the product data 
#     # Example of finding product descriptions
#     artworks = soup.find_all('div', class_='details')

#     art_list += artworks 

# Open the website
url = f'https://www.artnet.com/auctions/all-artworks/3'
driver.get(url)

time.sleep(5)

# Parse the page source using BeautifulSoup
soup = BeautifulSoup(driver.page_source, 'html.parser')

# Extract the product data 
# Example of finding product descriptions
artworks = soup.find_all('div', class_='details')

art_list += artworks 

# Close the WebDriver after scraping
driver.quit()

In [8]:
art_list

[<div class="details">
 <a href="/auctions/artists/steve-mccurry/impromptu-football-match-burma" ng-click="ensureRightPageWasSet(brick)">
 <ul>
 <li class="ng-binding"> Steve McCurry</li>
 <li class="ng-binding"><em class="ng-binding">Impromptu Footb...</em>, 1995</li>
 <!-- ngIf: brick.IsOpening && !brick.HideEstimate --><li class="ng-binding ng-scope" ng-if="brick.IsOpening &amp;&amp; !brick.HideEstimate">12,000—18,000 USD</li><!-- end ngIf: brick.IsOpening && !brick.HideEstimate -->
 <!-- ngIf: brick.IsOpening && brick.HideEstimate && !brick.isAcquire -->
 <!-- ngIf: brick.IsOnlyPurchaseNow && brick.IsInquiryNoPrice -->
 <!-- ngIf: brick.IsOpening && brick.isAcquire && brick.Estimate == '' -->
 <!-- ngIf: brick.IsOpening && brick.isAcquire && brick.Estimate != ''  && brick.HideEstimate -->
 <!-- ngIf: !brick.IsOpening && !brick.IsInquiryNoPrice -->
 <li class="ng-binding" ng-class="{'red' : brick.Remaining.Days &lt;= 0}" ng-hide="brick.IsOnlyPurchaseNow">
                           

In [9]:
artNetDict = {}

for index, art in enumerate(art_list, 1):
    artist_element = art.find("li", class_ = "ng-binding")
    if artist_element:
        artist_name = artist_element.text.strip()
    else:
        print(f"Missing artist name")
        continue

    piece_element = art.find("em", class_ = "ng-binding")
    if piece_element:
        piece_name = piece_element.text.strip()
    else:
        print(f"Missing piece name")
        continue

    price_element = art.find('li', class_="ng-binding ng-scope").text.strip()
    if "Bids" in price_element:
        price_text, bid_text = price_element.split(" (")
        price, currency = price_text.split(" ")
        price = float(price.replace(",", ""))
        bids = int(bid_text.replace("Bids)",""))
    elif "Bid" in price_element:
        price_text, bid_text = price_element.split(" (")
        price, currency = price_text.split(" ")
        price = float(price.replace(",", ""))
        bids = 1
    else:
        price_range, currency = price_element.split(" ")
        lower_bound, upper_bound = price_range.split("—")
        lower_bound = float(lower_bound.replace(",",""))
        upper_bound = float(upper_bound.replace(",",""))
        price = float(np.mean([lower_bound, upper_bound]))
        bids = 0

    expiration_element = art.find('li', {"ng-class": "{'red' : brick.Remaining.Days <= 0}"}).text.strip()
    if expiration_element:
        expiration, daysText, remainingText = expiration_element.split(" ")
        expiration = int(expiration)
    
    # print(artist_name, piece_name, price, currency, bids) -> to see if it all turned out okay
    
    artNetDict[index] = {}
    artNetDict[index]["Name"] = artist_name
    artNetDict[index]["Piece"] = piece_name
    artNetDict[index]["CurrentPrice"] = price
    artNetDict[index]["Currency"] = currency
    artNetDict[index]["NumBids"] = bids
    artNetDict[index]["DaysLeft"] = expiration

Missing artist name
Missing artist name
Missing artist name
Missing artist name
Missing artist name
Missing artist name
Missing artist name
Missing artist name
Missing artist name
Missing artist name
Missing artist name
Missing artist name
Missing artist name
Missing artist name
Missing artist name
Missing artist name
Missing artist name
Missing artist name
Missing artist name
Missing artist name
Missing artist name
Missing artist name
Missing artist name
Missing artist name
Missing artist name
Missing artist name
Missing artist name
Missing artist name
Missing artist name
Missing artist name
Missing artist name
Missing artist name
Missing artist name
Missing artist name
Missing artist name
Missing artist name
Missing artist name
Missing artist name
Missing artist name
Missing artist name
Missing artist name
Missing artist name
Missing artist name
Missing artist name
Missing artist name
Missing artist name
Missing artist name
Missing artist name
Missing artist name
Missing artist name


In [10]:
artNetDict

{1: {'Name': 'Steve McCurry',
  'Piece': 'Impromptu Footb...',
  'CurrentPrice': 15000.0,
  'Currency': 'USD',
  'NumBids': 0,
  'DaysLeft': 5},
 2: {'Name': 'John Chiara',
  'Piece': 'Hunterbrook at ...',
  'CurrentPrice': 10000.0,
  'Currency': 'USD',
  'NumBids': 0,
  'DaysLeft': 5},
 3: {'Name': 'Viviane Sassen',
  'Piece': 'Parasomnia',
  'CurrentPrice': 5000.0,
  'Currency': 'USD',
  'NumBids': 0,
  'DaysLeft': 5},
 4: {'Name': 'Steve McCurry',
  'Piece': 'Boat Covered in...',
  'CurrentPrice': 15000.0,
  'Currency': 'USD',
  'NumBids': 0,
  'DaysLeft': 5},
 5: {'Name': 'Edward Burtynsky',
  'Piece': 'Socar Oil Field...',
  'CurrentPrice': 10000.0,
  'Currency': 'USD',
  'NumBids': 0,
  'DaysLeft': 5},
 6: {'Name': 'Scott Peterman',
  'Piece': 'CR1',
  'CurrentPrice': 5000.0,
  'Currency': 'USD',
  'NumBids': 0,
  'DaysLeft': 5},
 13: {'Name': 'Emily Mason',
  'Piece': "Winter's Edge",
  'CurrentPrice': 46000.0,
  'Currency': 'USD',
  'NumBids': 2,
  'DaysLeft': 3},
 14: {'Name':

In [19]:
# col_names = ["Artist", "Name of Piece", "Current Price", "Currency", "Num of Bids", "Days Left"]
art_df = pd.DataFrame()

In [28]:
for key, value in artNetDict.items():
    row = pd.DataFrame({"Artist": [value["Name"]], "Name of Piece": [value["Piece"]], "Current Price": [value["CurrentPrice"]], "Currency": [value["Currency"]], "Num of Bids": [value["NumBids"]], "Days Left": [value["DaysLeft"]]})
    art_df = pd.concat([art_df, row], ignore_index= True)

art_df = art_df.sort_values(by=["Days Left"], ascending= True).reset_index()

In [29]:
art_df

,index,Artist,Name of Piece,Current Price,Currency,Num of Bids,Days Left
0,138,Emily Mason,Winter's Edge,46000.0,USD,2,3
1,81,Vivian Springford,Untitled (...,90000.0,USD,0,3
2,82,Janet Sobel,Untitled ...,22500.0,USD,0,3
3,83,Loie Hollowell,"Holy, Hole...",105000.0,USD,0,3
4,84,Robert Motherwell,Cor Anglais,125000.0,USD,0,3
...,...,...,...,...,...,...,...
193,94,Sally Mann,Landscape Missi...,15000.0,USD,1,5
194,95,Shirin Neshat,Untitled (Raptu...,25000.0,USD,0,5
195,96,Lalla Essaydi,Bullets Revisit...,25000.0,USD,0,5
196,69,Steve McCurry,Boat Covered in...,15000.0,USD,0,5
